In [2]:
!python -m pip install transformers torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 96.5 MB/s eta 0:00:00ta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.4/512.4 kB 60.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 kB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 62.0 MB/s eta 0:00:0000:01:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 770.0/770.0 kB 85.7 MB/s eta 0:00:00


In [ ]:
# !python -m pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import torchmetrics

In [5]:
import transformers

In [6]:
import torchmetrics

In [7]:
import traceback
import csv

import pandas as pd


def write_tsv_dataframe(filepath, dataframe):
    """
        Stores `DataFrame` as tsv file

        Parameters
        ----------
        filepath : str
            Path to tsv file
        dataframe : pd.DataFrame
            DataFrame to store

        Raises
        ------
        IOError
            if the file can't be opened
    """
    try:
        dataframe.to_csv(filepath, encoding='utf-8', sep='\t', index=False, header=True, quoting=csv.QUOTE_NONE)
    except IOError:
        traceback.print_exc()


In [8]:
def combine_columns(df_arguments, df_labels):
    """Combines the two `DataFrames` on column `Argument ID`"""
    return pd.merge(df_arguments, df_labels, on='Argument ID')


In [9]:
def split_arguments(df_arguments):
    """Splits `DataFrame` by column `Usage` into `train`-, `validation`-, and `test`-arguments"""
    train_arguments = df_arguments.loc[df_arguments['Usage'] == 'train'].drop(['Usage'], axis=1).reset_index(drop=True)
    valid_arguments = df_arguments.loc[df_arguments['Usage'] == 'validation'].drop(['Usage'], axis=1).reset_index(drop=True)
    test_arguments = df_arguments.loc[df_arguments['Usage'] == 'test'].drop(['Usage'], axis=1).reset_index(drop=True)
    
    return train_arguments, valid_arguments, test_arguments


In [10]:
def create_dataframe_head(argument_ids, model_name):
    """
        Creates `DataFrame` usable to append predictions to it

        Parameters
        ----------
        argument_ids : list[str]
            First column of the resulting DataFrame
        model_name : str
            Second column of DataFrame will contain the given model name

        Returns
        -------
        pd.DataFrame
            prepared DataFrame
    """
    df_model_head = pd.DataFrame(argument_ids, columns=['Argument ID'])
    df_model_head['Method'] = [model_name] * len(argument_ids)

    return df_model_head


In [11]:
import json
class MissingColumnError(AttributeError):
    """Error indicating that an imported DataFrame lacks necessary columns"""
    pass


In [12]:
def load_json_file(filepath):
    """Load content of json-file from `filepath`"""
    with open(filepath, 'r') as  json_file:
        return json.load(json_file)


In [13]:
def load_values_from_json(filepath):
    """Load values per level from json-file from `filepath`"""
    json_values = load_json_file(filepath)
    values = { "1":set(), "2":set(), "3":set(), "4a":set(), "4b":set() }
    for value in json_values["values"]:
        values["1"].add(value["name"])
        values["2"].add(value["level2"])
        for valueLevel3 in value["level3"]:
            values["3"].add(valueLevel3)
        for valueLevel4a in value["level4a"]:
            values["4a"].add(valueLevel4a)
        for valueLevel4b in value["level4b"]:
            values["4b"].add(valueLevel4b)
    values["1"] = sorted(values["1"])
    values["2"] = sorted(values["2"])
    values["3"] = sorted(values["3"])
    values["4a"] = sorted(values["4a"])
    values["4b"] = sorted(values["4b"])
    return values


In [14]:
def load_arguments_from_tsv(filepath, default_usage='test'):
    """
        Reads arguments from tsv file

        Parameters
        ----------
        filepath : str
            The path to the tsv file
        default_usage : str, optional
            The default value if the column "Usage" is missing

        Returns
        -------
        pd.DataFrame
            the DataFrame with all arguments

        Raises
        ------
        MissingColumnError
            if the required columns "Argument ID" or "Premise" are missing in the read data
        IOError
            if the file can't be read
        """
    try:
        dataframe = pd.read_csv(filepath, encoding='utf-8', sep='\t', header=0)
        if not {'Argument ID', 'Premise'}.issubset(set(dataframe.columns.values)):
            raise MissingColumnError('The argument "%s" file does not contain the minimum required columns [Argument ID, Premise].' % filepath)
        if 'Usage' not in dataframe.columns.values:
            dataframe['Usage'] = [default_usage] * len(dataframe)
        return dataframe
    except IOError:
        traceback.print_exc()
        raise


In [15]:
def load_labels_from_tsv(filepath, label_order):
    """
        Reads label annotations from tsv file

        Parameters
        ----------
        filepath : str
            The path to the tsv file
        label_order : list[str]
            The listing and order of the labels to use from the read data

        Returns
        -------
        pd.DataFrame
            the DataFrame with the annotations

        Raises
        ------
        MissingColumnError
            if the required columns "Argument ID" or names from `label_order` are missing in the read data
        IOError
            if the file can't be read
        """
    try:
        dataframe = pd.read_csv(filepath, encoding='utf-8', sep='\t', header=0)
        dataframe = dataframe[['Argument ID'] + label_order]
        return dataframe
    except IOError:
        traceback.print_exc()
        raise
    except KeyError:
        raise MissingColumnError('The file "%s" does not contain the required columns for its level.' % filepath)


In [16]:
import sys
import getopt
import os

In [17]:
model_dir = 'models'
data_dir = 'data'

In [18]:
if not os.path.exists(model_dir):
    os.makedirs(model_dir)


In [19]:
argument_filepath = os.path.join(data_dir, 'arguments.tsv')
value_json_filepath = os.path.join(data_dir, 'values.json')


In [20]:
df_arguments = load_arguments_from_tsv(argument_filepath, default_usage='train')

In [21]:
values = load_values_from_json(value_json_filepath)
num_labels_Lv2 = len(values['2'])


In [22]:
df_arguments.keys()

Index(['Argument ID', 'Part', 'Usage', 'Conclusion', 'Stance', 'Premise'], dtype='object')

In [24]:
# for ip in df_arguments['Argument ID']:
#   #print(df_arguments['Stance'][ip])
  
#   print(ip)


In [25]:
level =2
label_filepath = os.path.join(data_dir, 'labels-level{}.tsv'.format(str(level)))
df_labels = load_labels_from_tsv(label_filepath, values[str(level)])

In [26]:
a = df_labels.keys()
for key in df_labels.keys():
  print(len(df_labels[key]),key)

5270 Argument ID
5270 Achievement
5270 Benevolence: caring
5270 Benevolence: dependability
5270 Conformity: interpersonal
5270 Conformity: rules
5270 Face
5270 Hedonism
5270 Humility
5270 Power: dominance
5270 Power: resources
5270 Security: personal
5270 Security: societal
5270 Self-direction: action
5270 Self-direction: thought
5270 Stimulation
5270 Tradition
5270 Universalism: concern
5270 Universalism: nature
5270 Universalism: objectivity
5270 Universalism: tolerance


In [25]:

df_labels['Achievement'][0]

0

In [27]:
from typing import Dict, List

In [28]:
def generate_pairwise_input(dataset, labels, datatype):
    """
    TODO: group all premises and corresponding hypotheses and labels of the datapoints
    a datapoint as seen earlier is a dict of premis, hypothesis and label
    """
    #raise NotImplementedError
    premise=[]
    conclusion=[]
    stance=[]
    n_labels =labels.keys()
    n_labels = n_labels[1:]
    print(n_labels)
    label=[]
    
    n = len(dataset['Argument ID'])
    m = len(labels['Argument ID'])
    arguments = []
    print(n,m)
    for i in range(n):
        if dataset['Usage'][i]==datatype:
          premise.append(dataset['Premise'][i])
          conclusion.append(dataset['Conclusion'][i])
          stance.append(dataset['Stance'][i])
          arguments.append(dataset['Argument ID'][i])
    for i in range(m):
        if (labels['Argument ID'][i] in arguments):
          sent_label = []
          #print(i)
          for l in range(len(n_labels)):
              #print(n_labels[l])
              sent_label.append(int(labels[n_labels[l]][i]))
          label.append(sent_label)

    return premise, conclusion, stance, label

In [29]:
train_premises, train_conclusion, train_stance, train_labels = generate_pairwise_input(df_arguments, df_labels, 'train')
val_premises, val_conclusion, val_stance, val_labels = generate_pairwise_input(df_arguments, df_labels, 'validation')
test_premises, test_conclusion, test_stance, test_labels = generate_pairwise_input(df_arguments, df_labels, 'test')

Index(['Achievement', 'Benevolence: caring', 'Benevolence: dependability',
       'Conformity: interpersonal', 'Conformity: rules', 'Face', 'Hedonism',
       'Humility', 'Power: dominance', 'Power: resources',
       'Security: personal', 'Security: societal', 'Self-direction: action',
       'Self-direction: thought', 'Stimulation', 'Tradition',
       'Universalism: concern', 'Universalism: nature',
       'Universalism: objectivity', 'Universalism: tolerance'],
      dtype='object')
5270 5270
Index(['Achievement', 'Benevolence: caring', 'Benevolence: dependability',
       'Conformity: interpersonal', 'Conformity: rules', 'Face', 'Hedonism',
       'Humility', 'Power: dominance', 'Power: resources',
       'Security: personal', 'Security: societal', 'Self-direction: action',
       'Self-direction: thought', 'Stimulation', 'Tradition',
       'Universalism: concern', 'Universalism: nature',
       'Universalism: objectivity', 'Universalism: tolerance'],
      dtype='object')
5270 5

In [29]:
#Randomize them first
# train_premises, train_conclusion, train_stance, train_labels = generate_pairwise_input(df_arguments, df_labels)


In [30]:
# import random 
# random.seed(42)
# def randomize_data(premises, conclusion, stance, labels):
#   n = len(premises)
#   data = list(range(n))
#   random.shuffle(data)
#   train_premises = []
#   train_conclusion = []
#   train_stance = []
#   train_labels = []
#   for i in data:
#     train_premises.append(premises[i])
#     train_conclusion.append(conclusion[i])
#     train_stance.append(stance[i])
#     train_labels.append(labels[:][i])
#   return train_premises, train_conclusion, train_stance, train_labels


In [31]:
# train_premises, train_conclusion, train_stance, train_labels = randomize_data(train_premises, train_conclusion, train_stance, train_labels)

In [32]:
# val_premises = train_premises[-500:]
# val_conclusion = train_conclusion[-500:]
# val_stance = train_stance[-500:]
# val_labels = train_labels[:][-500:]

In [33]:
# train_premises = train_premises[:-500]
# train_conclusion = train_conclusion[:-500]
# train_stance = train_stance[:-500]
# train_labels = train_labels[:][:-500]

In [30]:
# Nothing to do for this class!
import torch
from transformers import BertModel
from transformers import AutoTokenizer
from typing import Dict, List

class BatchTokenizer:
    """Tokenizes and pads a batch of input sentences."""

    def __init__(self):
        """Initializes the tokenizer

        Args:
            pad_symbol (Optional[str], optional): The symbol for a pad. Defaults to "<P>".
        """
        self.hf_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
    
    def get_sep_token(self,):
        return self.hf_tokenizer.sep_token
    
    def __call__(self, prem_batch: List[str], hyp_batch: List[str], stance_batch: List[str]) -> List[List[str]]:
        """Uses the huggingface tokenizer to tokenize and pad a batch.

        We return a dictionary of tensors per the huggingface model specification.

        Args:
            batch (List[str]): A List of sentence strings

        Returns:
            Dict: The dictionary of token specifications provided by HuggingFace
        """
        # The HF tokenizer will PAD for us, and additionally combine 
        # The two sentences deimited by the [SEP] token.
        batch_len = len(prem_batch)
        #spaces = [" "]*batch_len
        conc_batch = [stance_batch[i]+" "+hyp_batch[i] for i in range(batch_len)]
        enc = self.hf_tokenizer(
            prem_batch,
            conc_batch,
            padding=True,
            return_token_type_ids=False,
            return_tensors='pt'
        )

        return enc
    

# HERE IS AN EXAMPLE OF HOW TO USE THE BATCH TOKENIZER
tokenizer = BatchTokenizer()
a = [["this is the premise.", "This is also a premise"], ["this is the hypothesis", "This is a second hypothesis"],["in favour of", "against"]]
x = tokenizer(*a)
print(x)
tokenizer.hf_tokenizer.batch_decode(x["input_ids"])



Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

{'input_ids': tensor([[  101,  2023,  2003,  1996, 18458,  1012,   102,  1999,  7927,  1997,
          2023,  2003,  1996, 10744,   102],
        [  101,  2023,  2003,  2036,  1037, 18458,   102,  2114,  2023,  2003,
          1037,  2117, 10744,   102,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]])}


['[CLS] this is the premise. [SEP] in favour of this is the hypothesis [SEP]',
 '[CLS] this is also a premise [SEP] against this is a second hypothesis [SEP] [PAD]']

In [31]:
def chunk(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[:][i:i + n]

def chunk_multi(lst1, lst2, lst3, n):
    for i in range(0, len(lst1), n):
        yield lst1[i: i + n], lst2[i: i + n], lst3[i: i + n]
        


In [32]:
sum=0
import numpy as np
# for i in range(5270):
#   sum += np.sum(np.array(train_labels[:][i]))
print(np.sum(np.array(train_labels)))
#print(sum)

14675


In [33]:
# against=0
# infavour = 0
# for i in range(4770):
#   if(train_stance[i]=='against'):
#     against +=1
#   elif(train_stance[i]=='in favor of'):
#     infavour += 1
#   else:
#     print(train_stance[i])


In [34]:
# Notice that since we use huggingface, we tokenize and
# encode in all at once!
batch_size=64
tokenizer = BatchTokenizer()
train_input_batches = [b for b in chunk_multi(train_premises, train_conclusion, train_stance, batch_size)]
# Tokenize + encode
train_input_batches = [tokenizer(*batch) for batch in train_input_batches]

In [35]:
val_input_batches = [b for b in chunk_multi(val_premises, val_conclusion, val_stance, batch_size)]
# Tokenize + encode
val_input_batches = [tokenizer(*batch) for batch in val_input_batches]


In [36]:
len(val_labels)

277

In [37]:
label_ids = ['Achievement', 'Benevolence: caring', 'Benevolence: dependability',
       'Conformity: interpersonal', 'Conformity: rules', 'Face', 'Hedonism',
       'Humility', 'Power: dominance', 'Power: resources',
       'Security: personal', 'Security: societal', 'Self-direction: action',
       'Self-direction: thought', 'Stimulation', 'Tradition',
       'Universalism: concern', 'Universalism: nature',
       'Universalism: objectivity', 'Universalism: tolerance']
l_ids = []
for l in label_ids:
    a =[l]
    l_ids.append(a)


In [ ]:
from transformers import BertTokenizer, BertModel
import torch
import numpy as np
embeddings = []
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
for i in range(20):
    input_ids = tokenizer(l_ids[i], return_tensors="pt")
    output = model(**input_ids)
    final_layer = output.last_hidden_state[:,0,:]
    #print(final_layer.shape)
    #hidden_shape = final_layer.shape
    #embed = torch.reshape(final_layer,(1,hidden_shape[1]) )
    embed = final_layer[0]
    embeddings.append(embed.detach().numpy())
embeddings = np.array(embeddings)
embeddings.shape

TypeError: ignored

In [ ]:
from nltk.cluster import KMeansClusterer
import nltk

def clustering_question(data,NUM_CLUSTERS = 5):
    kclusterer = KMeansClusterer(
        NUM_CLUSTERS, distance=nltk.cluster.util.cosine_distance,
        repeats=1000,avoid_empty_clusters=True)

    assigned_clusters = kclusterer.cluster(data, assign_clusters=True)

    return assigned_clusters

In [ ]:
clusters = clustering_question(embeddings)
clusters

[1, 3, 0, 0, 3, 1, 2, 1, 4, 4, 1, 2, 3, 3, 1, 1, 2, 0, 0, 0]

In [ ]:
cluster_labels =[[],[],[],[],[]]
cluster_ids = [[],[],[],[],[]]
for i in range(20):
    cluster_labels[clusters[i]].append(label_ids[i])
    cluster_ids[clusters[i]].append(i)
cluster_ids, cluster_labels

In [79]:
for i in range(20):
  print(i, label_ids[i])
print(np.nonzero(train_labels[0]), train_premises[0], train_labels[0])

0 Achievement
1 Benevolence: caring
2 Benevolence: dependability
3 Conformity: interpersonal
4 Conformity: rules
5 Face
6 Hedonism
7 Humility
8 Power: dominance
9 Power: resources
10 Security: personal
11 Security: societal
12 Self-direction: action
13 Self-direction: thought
14 Stimulation
15 Tradition
16 Universalism: concern
17 Universalism: nature
18 Universalism: objectivity
19 Universalism: tolerance
(array([11]),) if entrapment can serve to more easily capture wanted criminals, then why shouldn't it be legal? [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
# 0 Self-direction: thought
# 1 Self-direction: action	
# 2 Stimulation	
# 3 Hedonism	
# 4 Achievement	
# 5 Power: dominance	
# 6 Power: resources	
# 7 Face	
# 8 Security: personal	
# 9 Security: societal	
# 10 Tradition	
# 11 Conformity: rules	
# 12 Conformity: interpersonal	
# 13 Humility	
# 14 Benevolence: caring	
# 15 Benevolence: dependability	
# 16 Universalism: concern	
# 17 Universalism: nature	
# 18 Universalism: tolerance	
# 19 Universalism: objectivity

In [38]:
# Achievement, Face, Power: dominance, Power: resources [0, 5, 8, 9]


#Benevolence: caring, Benevolence: dependability, Humility, Universalism: concern [1,2,7, 16]


# Stimulation, Tradition, Self-direction: action, Self-direction: thought [14, 15, 12, 13]


# Conformity: interpersonal, Conformity: rules, Security: personal, Security: societal [3, 4, 10, 11]


# Hedonism, Universalism: nature, Universalism: objectivity, Universalism: tolerance [6, 17,18, 19]
# Note: This is just one possible way to group these elements. There may be other valid ways to do so.


clusters = [
    [0, 5, 8, 9],
    [1,2,7, 16],
    [14, 15, 12, 13],
    [3, 4, 10, 11],
    [6, 17,18, 19]
]


In [39]:
def encode_labels(labels: List[List[int]]) -> torch.FloatTensor:
    """Turns the batch of labels into a tensor

    Args:
        labels (List[List[int]]): List of all labels in the batch

    Returns:
        torch.FloatTensor: Tensor of all labels in the batch
    """
    
    extended_labels = []
    for i in range(len(labels)):
      elabels = [p for p in labels[i]]
      #Cluster 0
      for j in range(len(clusters)):
        clf = clusters[j]
        l = 0
        for k in clusters[j]:
          if(elabels[k]==1):
            l = 1
            break
        elabels.append(l)
      extended_labels.append(elabels)
    #print(len(extended_labels[0]), len(labels[0]))
    return torch.LongTensor(extended_labels)


In [40]:
check_labels = encode_labels(train_labels[0:10])
np.nonzero(train_labels[9]), np.nonzero(check_labels[9])

((array([12, 13, 15, 16]),),
 tensor([[12],
         [13],
         [15],
         [16],
         [21],
         [22]]))

In [41]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [42]:
train_label_batches = [b for b in chunk(train_labels, batch_size)]
train_label_batches = [encode_labels(batch) for batch in train_label_batches]

In [43]:
val_label_batches = [b for b in chunk(val_labels, batch_size)]
val_label_batches = [encode_labels(batch) for batch in val_label_batches]

In [69]:
val_label_batches[0][0]

tensor([1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1,
        1])

In [103]:
class GroupedClassifier(torch.nn.Module):
    def __init__(self, output_size: int, hidden_size: int):
        super().__init__()
        self.output_size = output_size
        self.hidden_size = hidden_size
        # Initialize BERT, which we use instead of a single embedding layer.
        self.bert = BertModel.from_pretrained("bert-base-uncased")
        # TODO [OPTIONAL]: Updating all BERT parameters can be slow and memory intensive. 
        # Freeze them if training is too slow. Notice that the learning
        # rate should probably be smaller in this case.
        # Uncommenting out the below 2 lines means only our classification layer will be updated.
        for param in self.bert.parameters():
            param.requires_grad = False
        self.bert_hidden_dimension = self.bert.config.hidden_size
        print(self.bert_hidden_dimension)
        # TODO: Add an extra hidden layer in the classifier, projecting
        #      from the BERT hidden dimension to hidden size.
        # TODO: Add a relu nonlinearity to be used in the forward method
        #      https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html
        self.hidden_layer = torch.nn.Linear(self.bert_hidden_dimension, 512)
        self.middle_layer1 = torch.nn.Linear(512, 64)
        self.middle_layer2 = torch.nn.Linear(512, 64)
        self.middle_layer3 = torch.nn.Linear(512, 64)
        self.middle_layer4 = torch.nn.Linear(512, 64)
        self.middle_layer5 = torch.nn.Linear(512, 64)
        #self.middle_layer6 = torch.nn.Linear(self.bert_hidden_dimension, 64)
        
        
        self.hidden_layer1 = torch.nn.Linear(64, 32)
        self.hidden_layer2 = torch.nn.Linear(64, 32)
        self.hidden_layer3 = torch.nn.Linear(64, 32)
        self.hidden_layer4 = torch.nn.Linear(64, 32)
        self.hidden_layer5 = torch.nn.Linear(64, 32)
        self.hidden_layer6 = torch.nn.Linear(64, 32)
        self.hidden_layer7 = torch.nn.Linear(64, 32)
        self.hidden_layer8 = torch.nn.Linear(64, 32)
        self.hidden_layer9 = torch.nn.Linear(64, 32)
        self.hidden_layer10 = torch.nn.Linear(64, 32)
        self.hidden_layer11 = torch.nn.Linear(64, 32)
        self.hidden_layer12 = torch.nn.Linear(64, 32)
        self.hidden_layer13 = torch.nn.Linear(64, 32)
        self.hidden_layer14 = torch.nn.Linear(64, 32)
        self.hidden_layer15 = torch.nn.Linear(64, 32)
        self.hidden_layer16 = torch.nn.Linear(64, 32)
        self.hidden_layer17 = torch.nn.Linear(64, 32)
        self.hidden_layer18 = torch.nn.Linear(64, 32)
        self.hidden_layer19 = torch.nn.Linear(64, 32)
        self.hidden_layer20 = torch.nn.Linear(64, 32)
        #self.hidden_layer2 = torch.nn.Linear(self.hidden_size, 32)
        #self.hidden_layer3 = torch.nn.Linear(128, 32)
        #self.hidden_layer4 = torch.nn.Linear(32, 8)
        self.relu = torch.nn.ReLU()
        self.classifier1 = torch.nn.Linear(32, 1)
        self.classifier2 = torch.nn.Linear(32, 1)
        self.classifier3 = torch.nn.Linear(32, 1)
        self.classifier4 = torch.nn.Linear(32, 1)
        self.classifier5 = torch.nn.Linear(32, 1)
        self.classifier6 = torch.nn.Linear(32, 1)
        self.classifier7 = torch.nn.Linear(32, 1)
        self.classifier8 = torch.nn.Linear(32, 1)
        self.classifier9 = torch.nn.Linear(32, 1)
        self.classifier10 = torch.nn.Linear(32, 1)
        self.classifier11 = torch.nn.Linear(32, 1)
        self.classifier12 = torch.nn.Linear(32, 1)
        self.classifier13 = torch.nn.Linear(32, 1)
        self.classifier14 = torch.nn.Linear(32, 1)
        self.classifier15 = torch.nn.Linear(32, 1)
        self.classifier16 = torch.nn.Linear(32, 1)
        self.classifier17 = torch.nn.Linear(32, 1)
        self.classifier18 = torch.nn.Linear(32, 1)
        self.classifier19 = torch.nn.Linear(32, 1)
        self.classifier20 = torch.nn.Linear(32, 1)
        
        self.classifier_middle1 = torch.nn.Linear(64, 1)
        self.classifier_middle2 = torch.nn.Linear(64, 1)
        self.classifier_middle3 = torch.nn.Linear(64, 1)
        self.classifier_middle4 = torch.nn.Linear(64, 1)
        self.classifier_middle5 = torch.nn.Linear(64, 1)
        #self.classifier = torch.nn.Linear(self.hidden_size, self.output_size)
        self.log_softmax = torch.nn.LogSoftmax(dim=2)

    def encode_text(
        self,
        symbols: Dict
    ) -> torch.Tensor:
        """Encode the (batch of) sequence(s) of token symbols with an LSTM.
            Then, get the last (non-padded) hidden state for each symbol and return that.

        Args:
            symbols (Dict): The Dict of token specifications provided by the HuggingFace tokenizer

        Returns:
            torch.Tensor: The final hiddens tate of the LSTM, which represents an encoding of
                the entire sentence
        """
        # First we get the contextualized embedding for each input symbol
        # We no longer need an LSTM, since BERT encodes context and 
        # gives us a single vector describing the sequence in the form of the [CLS] token.
        embedded = self.bert(**symbols)
        #print(embedded)
        #print("Embedded", embedded.pooler_output.shape, embedded.last_hidden_state.shape)
        # TODO: Get the [CLS] token using the `pooler_output` from 
        #      The BertModel output. See here: https://huggingface.co/docs/transformers/model_doc/bert#transformers.BertModel
        #      and check the returns for the forward method.
        # We want to return a tensor of the form batch_size x 1 x bert_hidden_dimension
        #raise NotImplementedError
        
        #pool_output_shape = embedded.pooler_output.shape
        #return torch.reshape(embedded.pooler_output,(pool_output_shape[0],1,pool_output_shape[1]) )
        last_hidden_state = embedded.last_hidden_state[:,0,:]
        hidden_shape = last_hidden_state.shape
        return torch.reshape(last_hidden_state,(hidden_shape[0],1,hidden_shape[1]) )

    def forward(
        self,
        symbols: Dict,
    ) -> torch.Tensor:
        """_summary_

        Args:
            symbols (Dict): The Dict of token specifications provided by the HuggingFace tokenizer

        Returns:
            torch.Tensor: _description_
        """
        encoded_sents = self.encode_text(symbols)
        #output = self.hidden_layer1(encoded_sents)
        #output = self.relu(output)
        #outputs = [self.hidden_layers[i](encoded_sents) for i in range(self.output_size)]
        #outputs = [self.relu(outputs[i].to(device)) for i in range(self.output_size)]
        #outputs = [self.classifiers[i](outputs[i].to(device)) for i in range(self.output_size)]
        # outputs = []
        # for i in range(self.output_size):
        #     output = self.hidden_layers[i](encoded_sents)
        #     output = self.relu(output)
        #     output = self.classifiers[i](output)
        #     output = torch.nn.Sigmoid()(output)
        #     outputs.append(output)
        enc = self.hidden_layer(encoded_sents)
        middle1 = self.middle_layer1(enc)
        middle1 = self.relu(middle1)
        middle_output1 = self.classifier_middle1(middle1)
        middle_output1 = torch.nn.Sigmoid()(middle_output1)
        
        middle2 = self.middle_layer2(enc)
        middle2 = self.relu(middle2)
        middle_output2 = self.classifier_middle2(middle2)
        middle_output2 = torch.nn.Sigmoid()(middle_output2)
        
        middle3 = self.middle_layer3(enc)
        middle3 = self.relu(middle3)
        middle_output3 = self.classifier_middle3(middle3)
        middle_output3 = torch.nn.Sigmoid()(middle_output3)
        
        middle4 = self.middle_layer4(enc)
        middle4 = self.relu(middle4)
        middle_output4 = self.classifier_middle4(middle4)
        middle_output4 = torch.nn.Sigmoid()(middle_output4)
        
        middle5 = self.middle_layer5(enc)
        middle5 = self.relu(middle5)
        middle_output5 = self.classifier_middle5(middle5)
        middle_output5 = torch.nn.Sigmoid()(middle_output5)
        
        output1 = self.hidden_layer1(middle1)
        output1 = self.relu(output1)
        output1 = self.classifier1(output1)
        output1 = torch.nn.Sigmoid()(output1)
        
        output2 = self.hidden_layer2(middle2)
        output2 = self.relu(output2)
        output2 = self.classifier2(output2)
        output2 = torch.nn.Sigmoid()(output2)
        
        output3 = self.hidden_layer3(middle2)
        output3 = self.relu(output3)
        output3 = self.classifier3(output3)
        output3 = torch.nn.Sigmoid()(output3)
        
        output4 = self.hidden_layer4(middle4)
        output4 = self.relu(output4)
        output4 = self.classifier4(output4)
        output4 = torch.nn.Sigmoid()(output4)
        
        output5 = self.hidden_layer5(middle4)
        output5 = self.relu(output5)
        output5 = self.classifier5(output5)
        output5 = torch.nn.Sigmoid()(output5)
        
        
        output6 = self.hidden_layer6(middle1)
        output6 = self.relu(output6)
        output6 = self.classifier6(output6)
        output6 = torch.nn.Sigmoid()(output6)
        
        
        output7 = self.hidden_layer7(middle5)
        output7 = self.relu(output7)
        output7 = self.classifier7(output7)
        output7 = torch.nn.Sigmoid()(output7)
        
        
        output8 = self.hidden_layer8(middle2)
        output8 = self.relu(output8)
        output8 = self.classifier8(output8)
        output8 = torch.nn.Sigmoid()(output8)
        
        output9 = self.hidden_layer9(middle1)
        output9 = self.relu(output9)
        output9 = self.classifier9(output9)
        output9 = torch.nn.Sigmoid()(output9)
        
        output10 = self.hidden_layer10(middle1)
        output10 = self.relu(output10)
        output10 = self.classifier10(output10)
        output10 = torch.nn.Sigmoid()(output10)
        
        output11 = self.hidden_layer11(middle4)
        output11 = self.relu(output11)
        output11 = self.classifier11(output11)
        output11 = torch.nn.Sigmoid()(output11)
        
        output12 = self.hidden_layer12(middle4)
        output12 = self.relu(output12)
        output12 = self.classifier12(output12)
        output12 = torch.nn.Sigmoid()(output12)
        
        output13 = self.hidden_layer13(middle3)
        output13 = self.relu(output13)
        output13 = self.classifier13(output13)
        output13 = torch.nn.Sigmoid()(output13)
        
        output14 = self.hidden_layer14(middle3)
        output14 = self.relu(output14)
        output14 = self.classifier14(output14)
        output14 = torch.nn.Sigmoid()(output14)
        
        output15 = self.hidden_layer15(middle3)
        output15 = self.relu(output15)
        output15 = self.classifier15(output15)
        output15 = torch.nn.Sigmoid()(output15)
        
        output16 = self.hidden_layer16(middle3)
        output16 = self.relu(output16)
        output16 = self.classifier16(output16)
        output16 = torch.nn.Sigmoid()(output16)
        
        output17 = self.hidden_layer17(middle2)
        output17 = self.relu(output17)
        output17 = self.classifier17(output17)
        output17 = torch.nn.Sigmoid()(output17)
        
        output18 = self.hidden_layer18(middle5)
        output18 = self.relu(output18)
        output18 = self.classifier18(output18)
        output18 = torch.nn.Sigmoid()(output18)
        
        output19 = self.hidden_layer19(middle5)
        output19 = self.relu(output19)
        output19 = self.classifier19(output19)
        output19 = torch.nn.Sigmoid()(output19)
        
        output20 = self.hidden_layer20(middle5)
        output20 = self.relu(output20)
        output20 = self.classifier20(output20)
        output20 = torch.nn.Sigmoid()(output20)
        outputs = torch.cat((output1, output2, output3, output4, output5, output6, output7, output8, output9, output10, output11, output12, output13, output14, output15, output16, output17, output18, output19, output20),2)
        group_outputs = torch.cat((middle_output1, middle_output2, middle_output3, middle_output4, middle_output5),2)
        return outputs, group_outputs

In [104]:
def predict(model: torch.nn.Module, sents: torch.Tensor) -> List:
    sents = sents.to(device)
    logits = model(sents)[0]
    res = []
    logitslen = len(logits)
    #print(logits[0].shape)
    for i in range(logitslen):
        datares = []
        for j in range(20):
            datares.append(logits[i][0][j] > 0.5)
        res.append(datares)
    return res


In [46]:
import numpy as np

from numpy import logical_and, sum as t_sum
def precision(predicted_labels, true_labels, which_label=1):
    """
    Precision is True Positives / All Positives Predictions
    """
    pred_which = np.array([pred == which_label for pred in predicted_labels])
    true_which = np.array([lab == which_label for lab in true_labels])
    denominator = t_sum(pred_which)
    if denominator:
        return t_sum(logical_and(pred_which, true_which))/denominator
    else:
        return 0.


def recall(predicted_labels, true_labels, which_label=1):
    """
    Recall is True Positives / All Positive Labels
    """
    pred_which = np.array([pred == which_label for pred in predicted_labels])
    true_which = np.array([lab == which_label for lab in true_labels])
    denominator = t_sum(true_which)
    if denominator:
        return t_sum(logical_and(pred_which, true_which))/denominator
    else:
        return 0.


def f1_score(
    predicted_labels: List[int],
    true_labels: List[int],
    which_label: int
):
    """
    F1 score is the harmonic mean of precision and recall
    """
    P = precision(predicted_labels, true_labels, which_label=which_label)
    R = recall(predicted_labels, true_labels, which_label=which_label)
    if P and R:
        return 2*P*R/(P+R)
    else:
        return 0.


def macro_f1(
    predicted_labels: List[int],
    true_labels: List[int],
    possible_labels: List[int]
):
    scores = [f1_score(predicted_labels, true_labels, l) for l in possible_labels]
    # Macro, so we take the uniform avg.
    print(scores)
    return sum(scores) / len(scores)

In [47]:
def f1Score_multiLabel(preds, labels):
    nLabels = 20
    relevants = [0]*20
    positives = [0]*20
    truePositives = [0]*20
    correct = [0]*20
    for i in range(len(preds)):
        for j in range(nLabels):
            if(preds[i][j]==1):
                positives[j] += 1
                if(labels[i][j]==1):
                    truePositives[j] += 1
    for i in range(len(preds)):
        for j in range(nLabels):
            if(preds[i][j]==labels[i][j]):
                correct[j] += 1
    
    for i in range(len(labels)):
        for j in range(nLabels):
            if(labels[i][j]==1):
                relevants[j] += 1
    
    precisions = []*nLabels
    recalls = []*nLabels
    f1Scores = []*nLabels
    accuracies = []*nLabels
    
    precision =0
    recall = 0
    f1 = 0
    #print(truePositives, positives, relevants)
    for i in range(nLabels):
        if(positives[i]>0):
            precision = truePositives[i]/positives[i]
        precisions.append(precision)
        if(relevants[i]>0):
            recall = truePositives[i]/relevants[i]
        recalls.append(recall)
        #print(precision,recall,i)
        if(precision>0 and recall>0):
            f1 = 2 * precision * recall / (precision + recall)
        f1Scores.append(f1)
        accuracies.append(correct[i]/len(preds))
    precision_mean = np.mean(precisions)
    recall_mean = np.mean(recalls)
    f1_mean = np.mean(f1Scores)
    accuracy = np.mean(accuracies)
    return f1_mean, precision_mean, recall_mean, accuracy, f1Scores, precisions, recalls, accuracies
    


In [97]:
import random
from tqdm import tqdm_notebook as tqdm
from torchmetrics.classification import MultilabelHammingDistance
def training_loop(
    num_epochs,
    train_features,
    train_labels,
    dev_sents,
    dev_labels,
    optimizer,
    scheduler,
    model,
):
    print("Training...")
    all_f1 = []
    all_P = []
    all_R = []
    all_L = []
    all_CELoss = []
    all_HMLoss = []
    all_acc = []
    loss_func = torch.nn.CrossEntropyLoss()
    hammingLoss20 = MultilabelHammingDistance(num_labels=20)
    batches = list(zip(train_features, train_labels))
    random.shuffle(batches)
    for i in range(num_epochs):
        losses = []
        #CELosses = []
        #HMLosses = []
        for features, labels in tqdm(batches):
            # Empty the dynamic computation graph
            features = features.to(device)
            labels = labels.float()
            labels = labels.to(device)
            optimizer.zero_grad()
            preds = model(features)
            local_loss = loss_func(preds[0].squeeze(1),labels[:,:20])
            group_loss = loss_func(preds[1].squeeze(1),labels[:,20:25])
            loss = local_loss + 4*group_loss
            #print("Preds ",preds.shape)
            #print("Labels ", labels.shape)
            #loss = loss_func(preds, labels)
            #loss2 = 20*hammingLoss(preds, labels)
            
            #loss = loss1 + loss2
            # Backpropogate the loss through our model
            loss.backward()
            optimizer.step()
            #CELosses.append(loss1.item())
            #HMLosses.append(loss2.item())
            losses.append(loss.item())
        
        #print(f"epoch {i}, loss: {np.sum(losses)/len(losses)}, HM loss: {np.sum(HMLosses)/len(HMLosses)}, CE loss: {np.sum(CELosses)/len(CELosses)}")
        print(f"epoch {i}, loss: {np.sum(losses)/len(losses)}")
        # Estimate the f1 score for the development set
        print("Evaluating dev...")
        all_preds = []
        all_labels = []
        for sents, labels in tqdm(zip(dev_sents, dev_labels), total=len(dev_sents)):
            sents = sents.to(device)
            pred = predict(model, sents)
            all_preds.extend(pred)
            all_labels.extend(list(labels))
        # #print(range(len(set(train_labels))))

        dev_f1, dev_P, dev_R, dev_acc, dev_all_f1, dev_all_P, dev_all_R, dev_all_acc = f1Score_multiLabel(all_preds, all_labels)
        print(f"Dev F1 {dev_f1},  Dev Precision {dev_P}, Dev Recall {dev_R}, Dev Accuracy {dev_acc}")
        all_f1.append(dev_all_f1)
        all_P.append(dev_all_P)
        all_R.append(dev_all_R)
        all_L.append(losses)
        #all_CELoss.append(CELosses)
        #all_HMLoss.append(HMLosses)
        all_acc.append(dev_all_acc)
        scheduler.step()
        #print(optimizer)
    # Return the trained model
    with open("all_f1_base.csv", 'ab') as abc:
        np.savetxt(abc, 
               all_f1,
               delimiter =", ", 
               fmt ='%s')
    with open("all_P_base.csv", 'ab') as abc:
        np.savetxt(abc, 
               all_P,
               delimiter =", ", 
               fmt ='%s')
    with open("all_R_base.csv", 'ab') as abc:
        np.savetxt(abc, 
               all_R,
               delimiter =", ", 
               fmt ='%s')
    with open("all_acc_base.csv", 'ab') as abc:
        np.savetxt(abc, 
               all_acc,
               delimiter =", ", 
               fmt ='%s')
    with open("all_L_base.csv", 'ab') as abc:
        np.savetxt(abc, 
               all_L,
               delimiter =", ", 
               fmt ='%s')
    """
    with open("all_CELoss_base.csv", 'ab') as abc:
        np.savetxt(abc, 
               all_CELoss,
               delimiter =", ", 
               fmt ='%s')
    with open("all_HMLoss_base.csv", 'ab') as abc:
        np.savetxt(abc, 
               all_CELoss,
               delimiter =", ", 
               fmt ='%s')
    """
    return model

In [98]:
from transformers.optimization import get_linear_schedule_with_warmup
epochs = 200
epoch_warmup = 40
# TODO: Find a good learning rate
LR = 1e-4

possible_labels = 20
model = GroupedClassifier(output_size=possible_labels, hidden_size=512)
model = model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), LR)
scheduler = get_linear_schedule_with_warmup(optimizer, epoch_warmup,epochs)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


768


In [ ]:

model =training_loop(
    epochs,
    train_input_batches,
    train_label_batches,
    val_input_batches,
    val_label_batches,
    optimizer,
    scheduler,
    model,
)

In [ ]:
torch.save(model, 'GroupClassifiers_CE.pt')

In [ ]:
print("Evaluating Test...")
all_preds = []
all_labels = []

test_input_batches = [b for b in chunk_multi(test_premises, test_conclusion, test_stance, batch_size)]
# Tokenize + encode
test_input_batches = [tokenizer(*batch) for batch in test_input_batches]


test_label_batches = [b for b in chunk(test_labels, batch_size)]
test_label_batches = [encode_labels(batch) for batch in test_label_batches]
for sents, labels in tqdm(zip(test_input_batches, test_label_batches), total=len(test_input_batches)):
    pred = predict(model, sents)
    all_preds.extend(pred)
    all_labels.extend(list(labels.numpy()))
# #print(range(len(set(train_labels))))

test_f1, test_P, test_R, test_acc, test_all_f1, test_all_P, test_all_R, test_all_acc = f1Score_multiLabel(all_preds, all_labels)
print(f"test F1 {test_f1},  test Precision {test_P}, test Recall {test_R}, test Accuracy {test_acc}")


0

In [105]:
### Trainining with CrossEnropy loss + Hamming loss
import random
from tqdm import tqdm_notebook as tqdm
from torchmetrics.classification import MultilabelHammingDistance
def training_loop(
    num_epochs,
    train_features,
    train_labels,
    dev_sents,
    dev_labels,
    optimizer,
    scheduler,
    model,
):
    print("Training...")
    all_f1 = []
    all_P = []
    all_R = []
    all_L = []
    all_CELoss = []
    all_HMLoss = []
    all_acc = []
    loss_func = torch.nn.CrossEntropyLoss()
    hammingLoss20 = MultilabelHammingDistance(num_labels=20).to(device)
    hammingLoss5 = MultilabelHammingDistance(num_labels=5).to(device)
    batches = list(zip(train_features, train_labels))
    random.shuffle(batches)
    for i in range(num_epochs):
        losses = []
        CELosses = []
        HMLosses = []
        for features, labels in tqdm(batches):
            # Empty the dynamic computation graph
            features = features.to(device)
            labels = labels.float()
            labels = labels.to(device)
            optimizer.zero_grad()
            preds0, preds1 = model(features)
            local_loss1 = loss_func(preds0.squeeze(1),labels[:,:20])
            group_loss1 = loss_func(preds1.squeeze(1),labels[:,20:25])
            loss1 = local_loss1 + 4*group_loss1

            # preds0 = preds[0].to(device)
            # preds1 = preds[1].to(device)
            local_loss2 = hammingLoss20(preds0.squeeze(1),labels[:,:20])
            # print(preds1.shape)
            group_loss2 = hammingLoss5(preds1.squeeze(1),labels[:,20:25])
            loss2 = local_loss2 + 4*group_loss2

            loss = loss1 + 20*loss2
            #print("Preds ",preds.shape)
            #print("Labels ", labels.shape)
            #loss = loss_func(preds, labels)
            #loss2 = 20*hammingLoss(preds, labels)
            
            #loss = loss1 + loss2
            # Backpropogate the loss through our model
            loss.backward()
            optimizer.step()
            CELosses.append(loss1.item())
            HMLosses.append(loss2.item())
            losses.append(loss.item())
        
        print(f"epoch {i}, loss: {np.sum(losses)/len(losses)}, HM loss: {np.sum(HMLosses)/len(HMLosses)}, CE loss: {np.sum(CELosses)/len(CELosses)}")
        #print(f"epoch {i}, HM loss: {np.sum(losses)/len(losses)}")
        # Estimate the f1 score for the development set
        print("Evaluating dev...")
        all_preds = []
        all_labels = []
        for sents, labels in tqdm(zip(dev_sents, dev_labels), total=len(dev_sents)):
            sents = sents.to(device)
            pred = predict(model, sents)
            all_preds.extend(pred)
            all_labels.extend(list(labels))
        # #print(range(len(set(train_labels))))

        dev_f1, dev_P, dev_R, dev_acc, dev_all_f1, dev_all_P, dev_all_R, dev_all_acc = f1Score_multiLabel(all_preds, all_labels)
        print(f"Dev F1 {dev_f1},  Dev Precision {dev_P}, Dev Recall {dev_R}, Dev Accuracy {dev_acc}")
        all_f1.append(dev_all_f1)
        all_P.append(dev_all_P)
        all_R.append(dev_all_R)
        all_L.append(losses)
        all_CELoss.append(CELosses)
        all_HMLoss.append(HMLosses)
        all_acc.append(dev_all_acc)
        scheduler.step()
        #print(optimizer)
    # Return the trained model
    with open("all_f1_base.csv", 'ab') as abc:
        np.savetxt(abc, 
               all_f1,
               delimiter =", ", 
               fmt ='%s')
    with open("all_P_base.csv", 'ab') as abc:
        np.savetxt(abc, 
               all_P,
               delimiter =", ", 
               fmt ='%s')
    with open("all_R_base.csv", 'ab') as abc:
        np.savetxt(abc, 
               all_R,
               delimiter =", ", 
               fmt ='%s')
    with open("all_acc_base.csv", 'ab') as abc:
        np.savetxt(abc, 
               all_acc,
               delimiter =", ", 
               fmt ='%s')
    with open("all_L_base.csv", 'ab') as abc:
        np.savetxt(abc, 
               all_L,
               delimiter =", ", 
               fmt ='%s')
    
    with open("all_CELoss_base.csv", 'ab') as abc:
        np.savetxt(abc, 
               all_CELoss,
               delimiter =", ", 
               fmt ='%s')
    with open("all_HMLoss_base.csv", 'ab') as abc:
        np.savetxt(abc, 
               all_CELoss,
               delimiter =", ", 
               fmt ='%s')
    
    return model

In [106]:
from transformers.optimization import get_linear_schedule_with_warmup
epochs = 200
epoch_warmup = 40
# TODO: Find a good learning rate
LR = 1e-5

possible_labels = 20
model = GroupedClassifier(output_size=possible_labels, hidden_size=512)
model = model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), LR)
scheduler = get_linear_schedule_with_warmup(optimizer, epoch_warmup,epochs)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


768


In [ ]:

model =training_loop(
    epochs,
    train_input_batches,
    train_label_batches,
    val_input_batches,
    val_label_batches,
    optimizer,
    #scheduler,
    model,
)

Training...


/tmp/ipykernel_457/3952115150.py:31: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for features, labels in tqdm(batches):


  0%|          | 0/67 [00:00<?, ?it/s]

epoch 0, loss: 67.1073558294951, HM loss: 2.021245333685804, CE loss: 26.682449383522147
Evaluating dev...


/tmp/ipykernel_457/3952115150.py:69: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for sents, labels in tqdm(zip(dev_sents, dev_labels), total=len(dev_sents)):


...

  0%|          | 0/67 [00:00<?, ?it/s]

epoch 29, loss: 55.81236045040301, HM loss: 1.5514342393448104, CE loss: 24.783675478465522
Evaluating dev...


  0%|          | 0/5 [00:00<?, ?it/s]

Dev F1 0.40458991005462713,  Dev Precision 0.3219801963092329, Dev Recall 0.276021295654184, Dev Accuracy 0.7550541516245487


  0%|          | 0/67 [00:00<?, ?it/s]

In [ ]:
print("Evaluating Test...")
all_preds = []
all_labels = []

test_input_batches = [b for b in chunk_multi(test_premises, test_conclusion, test_stance, batch_size)]
# Tokenize + encode
test_input_batches = [tokenizer(*batch) for batch in test_input_batches]


test_label_batches = [b for b in chunk(test_labels, batch_size)]
test_label_batches = [encode_labels(batch) for batch in test_label_batches]
for sents, labels in tqdm(zip(test_input_batches, test_label_batches), total=len(test_input_batches)):
    pred = predict(model, sents)
    all_preds.extend(pred)
    all_labels.extend(list(labels.numpy()))
# #print(range(len(set(train_labels))))

test_f1, test_P, test_R, test_acc, test_all_f1, test_all_P, test_all_R, test_all_acc = f1Score_multiLabel(all_preds, all_labels)
print(f"test F1 {test_f1},  test Precision {test_P}, test Recall {test_R}, test Accuracy {test_acc}")


In [68]:
torch.save(model,"Grouping_Hybrid_loss.pt")